In [1]:
from hana_ml import dataframe as hdf
import pandas as pd

In [2]:
myconn=hdf.ConnectionContext(userkey='DevChallenger')
print("SAP HANA DB version: ", myconn.hana_version())

SAP HANA DB version:  4.00.000.00.1678185420 (fa/CE2022.42)


In [3]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

2023-03-17 07:44:29.639000


In [4]:
mycursor = myconn.connection.cursor()

In [5]:
# mycursor.execute('''
# CREATE TABLE "SEASONS" (
#   "ID" INT PRIMARY KEY, 
#   "DESC" VARCHAR(12)
# )
# ''');

In [6]:
# mycursor.execute('TRUNCATE TABLE "SEASONS"');

# records_inserted=mycursor.executemany(operation='INSERT INTO "SEASONS" ("ID", "DESC") VALUES (?, ?)',
#                                      list_of_parameters=
#                                       ((1, "WINTER"),
#                                       (2, "SPRING"),
#                                       (3, "SUMMER"),
#                                       (4, "FALL"))
#                                      )
# print("Records inserted: ", sum(records_inserted))

In [7]:
mycursor.execute('SELECT * FROM "SEASONS"')
#Get the first record
result=mycursor.fetchone()
print("the first record", result)

#Get the second record
result=mycursor.fetchone()
print("the second record", result)

#Get the rest of records
result=mycursor.fetchall()
print("the rest of records", result)

the first record (1, 'WINTER')
the second record (2, 'SPRING')
the rest of records [(3, 'SUMMER'), (4, 'FALL')]


In [8]:
print(f"Current database schema: {myconn.get_current_schema()}")

Current database schema: DEVCHALLENGER


In [9]:
mycursor.execute(f'''
SELECT TABLE_NAME
FROM M_TABLES
WHERE SCHEMA_NAME='{myconn.get_current_schema()}'
''')

#Get all records
result=mycursor.fetchall()
print(f"{len(result)} table(s) in the schema {myconn.get_current_schema()}: {[x[0] for x in result]}")

5 table(s) in the schema DEVCHALLENGER: ['CML_WASTE_WATER_VERTICES', 'BIKES', 'SEASONS', 'CML_WASTE_WATER_EDGES', 'MONTHS']


In [10]:
# myconn.drop_table("MONTHS")
# myconn.create_table("MONTHS", table_structure={"ID":"INT", "DESC":"VARCHAR(10)"})

In [11]:
myconn.get_tables()

,TABLE_NAME
0,CML_WASTE_WATER_VERTICES
1,BIKES
2,CML_WASTE_WATER_EDGES
3,SEASONS
4,MONTHS


In [12]:
months_df=pd.DataFrame(
    {"ID": range(1,13),
     "DESC": ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
    })

In [13]:
display(months_df)

,ID,DESC
0,1,JAN
1,2,FEB
2,3,MAR
3,4,APR
4,5,MAY
5,6,JUN
6,7,JUL
7,8,AUG
8,9,SEP
9,10,OCT


In [14]:
months_hdf = hdf.create_dataframe_from_pandas(connection_context=myconn,
                                      pandas_df=months_df,
                                      table_name='MONTHS',
                                      primary_key='ID',
                                      force=True
                                     )

100%|██████████| 1/1 [00:00<00:00,  1.51it/s]


In [15]:
type(months_hdf)

hana_ml.dataframe.DataFrame

In [16]:
print(months_hdf.select_statement)

SELECT * FROM "MONTHS"


In [17]:
mycursor.execute(months_hdf.select_statement)

#Get all records
result=mycursor.fetchall()

#Display returned records
print(f"All {len(result)} records: {result}")

All 12 records: [(1, 'JAN'), (2, 'FEB'), (3, 'MAR'), (4, 'APR'), (5, 'MAY'), (6, 'JUN'), (7, 'JUL'), (8, 'AUG'), (9, 'SEP'), (10, 'OCT'), (11, 'NOV'), (12, 'DEC')]


In [18]:
months_hdf.collect()

,ID,DESC
0,1,JAN
1,2,FEB
2,3,MAR
3,4,APR
4,5,MAY
5,6,JUN
6,7,JUL
7,8,AUG
8,9,SEP
9,10,OCT


In [19]:
type(months_df)

pandas.core.frame.DataFrame

In [20]:
url="https://raw.githubusercontent.com/SAP-samples/hana-ml-samples/main/Python-API/pal/datasets/bike.csv"
bikes_df = pd.io.parsers.read_csv(url)

In [21]:
display(bikes_df)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,hum,windspeed,cnt,days_since_2011
0,WINTER,2011,JAN,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,8.175849,80.5833,10.749882,985,0
1,WINTER,2011,JAN,NO HOLIDAY,SUN,NO WORKING DAY,MISTY,9.083466,69.6087,16.652113,801,1
2,WINTER,2011,JAN,NO HOLIDAY,MON,WORKING DAY,GOOD,1.229108,43.7273,16.636703,1349,2
3,WINTER,2011,JAN,NO HOLIDAY,TUE,WORKING DAY,GOOD,1.400000,59.0435,10.739832,1562,3
4,WINTER,2011,JAN,NO HOLIDAY,WED,WORKING DAY,GOOD,2.666979,43.6957,12.522300,1600,4
...,...,...,...,...,...,...,...,...,...,...,...,...
726,WINTER,2012,DEC,NO HOLIDAY,THU,WORKING DAY,MISTY,3.945849,65.2917,23.458911,2114,726
727,WINTER,2012,DEC,NO HOLIDAY,FRI,WORKING DAY,MISTY,3.906651,59.0000,10.416557,3095,727
728,WINTER,2012,DEC,NO HOLIDAY,SAT,NO WORKING DAY,MISTY,3.906651,75.2917,8.333661,1341,728
729,WINTER,2012,DEC,NO HOLIDAY,SUN,NO WORKING DAY,GOOD,4.024151,48.3333,23.500518,1796,729


In [ ]:
bikes_hdf = hdf.create_dataframe_from_pandas(connection_context=myconn,
                                      pandas_df=bikes_df,
                                      table_name='BIKES',
                                      primary_key='days_since_2011',
                                      force=True
                                     )

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
type(bikes_hdf)

In [ ]:
print(bikes_hdf.select_statement)

In [ ]:
bikes_hdf.collect().head(n=10)

In [ ]:
bikes_hdf.collect()["season"].unique().tolist()